In [10]:
def get_data(path, OB, filter_bad = False):

    fac = 10
    # print(int(OB[2:]))
    # if int(OB[2:]) > 5:
    #     fac = 1


    f = fits.open("%s/UVB%s.fits"%(path, OB))
    # print(f[1].columns)
    wl = fac*f[1].data.field("WAVE").flatten()
    q = f[1].data.field("QUAL").flatten()
    mask_wl = (wl > 3200) & (wl < 5600)
    mask_qual = ~q.astype("bool")
    flux = interpolate.interp1d(wl[mask_qual], f[1].data.field("FLUX").flatten()[mask_qual], bounds_error=False, fill_value=0)
    error = interpolate.interp1d(wl[mask_qual], f[1].data.field("ERR").flatten()[mask_qual], bounds_error=False, fill_value=0)
    wl_plot = wl[mask_wl]
    flux = flux(wl_plot)
    error = error(wl_plot)
    wl_UVB = wl_plot
    flux_UVB = flux
    error_UVB = error

    f = fits.open("%s/VIS%s.fits"%(path, OB))
    wl = fac*f[1].data.field("WAVE").flatten()
    q = f[1].data.field("QUAL").flatten()
    try:
        t = f[1].data.field("TRANS").flatten()
    except:
        t = np.ones_like(f[1].data.field("QUAL").flatten())
    mask_wl = (wl > 5600) & (wl < 10200)
    mask_qual = ~q.astype("bool")

    flux = interpolate.interp1d(wl[mask_qual], f[1].data.field("FLUX").flatten()[mask_qual], bounds_error=False, fill_value=0)
    error = interpolate.interp1d(wl[mask_qual], f[1].data.field("ERR").flatten()[mask_qual], bounds_error=False, fill_value=0)
    wl_plot = wl[mask_wl]
    flux = flux(wl_plot)/t[mask_wl]
    error = error(wl_plot)/t[mask_wl]
    wl_VIS = wl_plot
    flux_VIS = flux
    error_VIS = error

    f = fits.open("%s/NIR%s.fits"%(path, OB))
    try:
        f_nod = fits.open("%s/NIR%s_NOD.fits"%(path, OB))
    except:
        f_nod = fits.open("%s/NIR%s.fits"%(path, OB))
    wl = fac*f[1].data.field("WAVE").flatten()
    f[1].data["FLUX"] = np.concatenate((f[1].data["FLUX"].flatten()[(wl <= 22500)], f_nod[1].data["FLUX"].flatten()[(wl > 22500)]))
    f[1].data["ERR"] = np.concatenate((f[1].data["ERR"].flatten()[(wl <= 22500)], f_nod[1].data["ERR"].flatten()[(wl > 22500)]))

    q = f[1].data.field("QUAL").flatten()
    try:
        t = f[1].data.field("TRANS").flatten()
    except:
        t = np.ones_like(f[1].data.field("QUAL").flatten())
    mask_wl = (wl > 10200)
    mask_qual = ~q.astype("bool")
    flux = interpolate.interp1d(wl[mask_qual], f[1].data.field("FLUX").flatten()[mask_qual], bounds_error=False, fill_value=0)
    error = interpolate.interp1d(wl[mask_qual], f[1].data.field("ERR").flatten()[mask_qual], bounds_error=False, fill_value=0)
    wl_plot = wl[mask_wl]
    flux = flux(wl_plot)/t[mask_wl]
    error = error(wl_plot)/t[mask_wl]
    wl_NIR = wl_plot
    flux_NIR = flux
    error_NIR = error

    print(min(wl_UVB))


    wl = np.concatenate((wl_UVB, wl_VIS, wl_NIR))
    flux = np.concatenate((flux_UVB, flux_VIS, flux_NIR))
    error = np.concatenate((error_UVB, error_VIS, error_NIR))

    if filter_bad:

        wl, flux, error = filter_bad_values(wl, flux, error)

    return wl, flux, error

from astropy.io import fits
from scipy import interpolate
import numpy as np
from scipy.signal import medfilt


def filter_bad_values(wl, flux, error):
    medfilter = medfilt(flux, 501)
    mask = np.logical_and(abs(flux - medfilter) < 3*error, ~np.isnan(flux))
    f = interpolate.interp1d(wl[mask], flux[mask], bounds_error=False, fill_value = np.nan)
    g = interpolate.interp1d(wl[mask], error[mask], bounds_error=False, fill_value = np.nan)
    return wl, f(wl), g(wl)

In [4]:

OB='OB1'
path = './spectra/'
wl2, flux2, error2 = get_data(path, OB, filter_bad=True)
z = 0.009783
wl2, flux2, error2 = wl2/(1.+z), flux2*(1.+z), np.sqrt((error2**2.)*(1.+z))
np.savetxt('epoch1.txt', [wl2,flux2,error2])

OB='OB2'
path = './spectra/'
wl2, flux2, error2 = get_data(path, OB, filter_bad=True)
z = 0.009783
wl2, flux2, error2 = wl2/(1.+z), flux2*(1.+z), np.sqrt((error2**2.)*(1.+z))
np.savetxt('epoch2.txt', [wl2,flux2,error2])

OB='OB3'
path = './spectra/'
wl2, flux2, error2 = get_data(path, OB, filter_bad=True)
z = 0.009783
wl2, flux2, error2 = wl2/(1.+z), flux2*(1.+z), np.sqrt((error2**2.)*(1.+z))
np.savetxt('epoch3.txt', [wl2,flux2,error2])

OB='OB4'
path = './spectra/'
wl2, flux2, error2 = get_data(path, OB, filter_bad=True)
z = 0.009783
wl2, flux2, error2 = wl2/(1.+z), flux2*(1.+z), np.sqrt((error2**2.)*(1.+z))
np.savetxt('epoch4.txt', [wl2,flux2,error2])

OB='OB5'
path = './spectra/'
wl2, flux2, error2 = get_data(path, OB, filter_bad=True)
z = 0.009783
wl2, flux2, error2 = wl2/(1.+z), flux2*(1.+z), np.sqrt((error2**2.)*(1.+z))
np.savetxt('epoch5.txt', [wl2,flux2,error2])

3200.079
3200.043
3200.085
3200.052
3200.112


In [12]:
OB='OB6'
path = './spectra/new'
wl2, flux2, error2 = get_data(path, OB, filter_bad=True)
z = 0.009783
wl2, flux2, error2 = wl2/(1.+z), flux2*(1.+z), np.sqrt((error2**2.)*(1.+z))
np.savetxt('epoch6.txt', [wl2,flux2,error2])

OB='OB7'
path = './spectra/new'
wl2, flux2, error2 = get_data(path, OB, filter_bad=True)
z = 0.009783
wl2, flux2, error2 = wl2/(1.+z), flux2*(1.+z), np.sqrt((error2**2.)*(1.+z))
np.savetxt('epoch7.txt', [wl2,flux2,error2])

OB='OB8'
path = './spectra/new'
wl2, flux2, error2 = get_data(path, OB, filter_bad=True)
z = 0.009783
wl2, flux2, error2 = wl2/(1.+z), flux2*(1.+z), np.sqrt((error2**2.)*(1.+z))
np.savetxt('epoch8.txt', [wl2,flux2,error2])

OB='OB9'
path = './spectra/new'
wl2, flux2, error2 = get_data(path, OB, filter_bad=True)
z = 0.009783
wl2, flux2, error2 = wl2/(1.+z), flux2*(1.+z), np.sqrt((error2**2.)*(1.+z))
np.savetxt('epoch9.txt', [wl2,flux2,error2])

3200.1490000000003
3200.115
3200.035
3200.09


In [ ]:
OB='OB1'
path = './spectra_filtered/'
wl2, flux2, error2 = get_data(path, OB)
z = 0.009783
wl2, flux2, error2 = wl2/(1.+z), flux2*(1.+z), np.sqrt((error2**2.)*(1.+z))
np.savetxt('epoch1.txt', [wl2,flux2,error2])

OB='OB2'
path = './spectra_filtered/'
wl2, flux2, error2 = get_data(path, OB)
z = 0.009783
wl2, flux2, error2 = wl2/(1.+z), flux2*(1.+z), np.sqrt((error2**2.)*(1.+z))
np.savetxt('epoch2.txt', [wl2,flux2,error2])

OB='OB3'
path = './spectra_filtered/'
wl2, flux2, error2 = get_data(path, OB)
z = 0.009783
wl2, flux2, error2 = wl2/(1.+z), flux2*(1.+z), np.sqrt((error2**2.)*(1.+z))
np.savetxt('epoch3.txt', [wl2,flux2,error2])

OB='OB4'
path = './spectra_filtered/'
wl2, flux2, error2 = get_data(path, OB)
z = 0.009783
wl2, flux2, error2 = wl2/(1.+z), flux2*(1.+z), np.sqrt((error2**2.)*(1.+z))
np.savetxt('epoch4.txt', [wl2,flux2,error2])

OB='OB5'
path = './spectra_filtered/'
wl2, flux2, error2 = get_data(path, OB)
z = 0.009783
wl2, flux2, error2 = wl2/(1.+z), flux2*(1.+z), np.sqrt((error2**2.)*(1.+z))
np.savetxt('epoch5.txt', [wl2,flux2,error2])

3200.1490000000003


In [40]:
OB='OB3'
path = './spectra/'
f = fits.open("%s/NIR%s.fits"%(path, OB))